In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import Dict, List
from toolkit.tire_model.tire_model_pacejka_2010 import *
from toolkit.tire_model.tire_model_fitting import *
from toolkit.tire_model.ttc_loader import *
from toolkit.tire_model.tire_fitting_masks import *
from toolkit.tire_model.tire_model_utils import *

In [ ]:
# or from a existing set of parameters
tire_model_1 = tire_model_from_arr(H_R20_18X6_7)
tire_model_2 = tire_model_from_arr(H_R20_18X6_6)
tire_model_og = readTIR(make_path('./Data/TTCData/TIR_Templates/FSAE_Defaults.tir'))

In [ ]:
# Generate graphs to check smoothness of the fit
# sweep_SA([tire_model, tire_model_2, tire_model_3, tire_model_og, tire_model_4], "Slip Angle")
sweep_SA([tire_model_1], "Slip Angle") # , tire_model_2

In [ ]:
# Generate graphs to check smoothness of the fit
# sweep_SR([tire_model, tire_model_2, tire_model_3, tire_model_og, tire_model_4], "Slip Ratio")
sweep_SR([tire_model_1], "Slip Ratio") # , tire_model_2

In [ ]:
params = tire_model_1.dump_params()
paramss = tire_model_og.dump_params()
paramsss = tire_model_2.dump_params()
for i in FX_MASK_IND:#range(len(params)): # FY_MASK_IND_PRESS:#
        print(f"{i}\t{params[i]:.4f}\t{paramsss[i]:.4f}\t{LABELS[i]}\t{NAMES[i]}") # \t{paramsss[i]:.4f} \t{params[i]:.4f}

In [ ]:
result, value, c, ins = tire_model_2.coefficientCheck()

In [ ]:
print(f"Cx error: {result.Cx}")
print(f"Dx error: {np.any(result.Dx)}")
print(f"Ex error: {np.any(result.Ex)}")
print(f"Cy error: {result.Cy}")
print(f"Ey error: {np.any(result.Ey)}")
print(f"Bt error: {np.any(result.Bt)}")
print(f"Ct error: {result.Ct}")
print(f"Et error: {np.any(result.Et)}")

In [ ]:
# Get the ttc data for the tire you want to fit
cornering_raw, drive_brake_raw, name = get_LC0_18x6_6_runs()
# create all the boring lists and stuff

params_list: List = []
error_list: List = []
combi_runs = None
cornering = None
drive_brake = None
for df_raw in cornering_raw:
    df = df_raw#flip_and_merge(df_raw)
    combi_runs = merge(df, combi_runs)
    cornering = merge(df, cornering)

for df_raw in drive_brake_raw:
    df = df_raw#flip_and_merge(df_raw)
    combi_runs = merge(df, combi_runs)
    drive_brake = merge(df, drive_brake)

In [ ]:
df = remove_time_gaps(filter_vel(combi_runs))
df = df[df.TSTC > 50]
fx_1_error, fy_1_error, mz_1_error = get_model_error(tire_model_1, df)
fx_2_error, fy_2_error, mz_2_error = get_model_error(tire_model_2, df)
fig = go.Figure()
fig.add_trace(go.Histogram(x=fx_1_error, histnorm='probability density', name='Fx 1'))
fig.add_trace(go.Histogram(x=fy_1_error, histnorm='probability density', name='Fy 1'))
fig.add_trace(go.Histogram(x=mz_1_error, histnorm='probability density', name='Mz 1'))
fig.add_trace(go.Histogram(x=fx_2_error, histnorm='probability density', name='Fx 2'))
fig.add_trace(go.Histogram(x=fy_2_error, histnorm='probability density', name='Fy 2'))
fig.add_trace(go.Histogram(x=mz_2_error, histnorm='probability density', name='Mz 2'))
# put graph key in bottom right
fig.update_layout(barmode='overlay', legend=dict(orientation="h", yanchor="bottom", y=0.02, xanchor="right", x=1))
fig.update_layout(template="plotly_dark", margin=dict(l=0, r=0, b=0, t=0), xaxis_title='Error (N)', yaxis_title='Probability Density')
fig.show()